In [1]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span,ColorBar, LogColorMapper
from bokeh.palettes import viridis



# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.1
T7_DNA.lc =0.1

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0
Repressor_DNA.lc = 0


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [48]:
#experiment 
T7_DNA.ic = 0.1
T7_DNA.lc = 0.1
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


#######create plot#############
p = bokeh.plotting.figure(
    plot_width=1000,
    plot_height=500,
    x_axis_label="t",
    y_axis_type="linear",
    title="Resource"
)


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition



DR = 0.2
color_num = 0

condition_low_log = -2
condition_high_log = 3


for Resource in np.logspace(condition_low_log,condition_high_log,50):
    color_num += 1
    parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)

    R.ic = Resource
    R.lc = Resource

    #initial value
    molecules_0 = [None]*len(molecules_list) #create empty list

    for i in range(0,len(molecules_list)):
        molecules_0[i] = molecules_list[i].ic
    #stage 1 kick start
    result_all=[molecules_0] #initial condition

    #kick start
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [T7,R]
    result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all,DR)


    #self regeneration 
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(4,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all,DR)



    result_all = np.resize(result_all,(121,10))
    for i in range(0,len(molecules_list)):
            (molecules_list[i]).result = result_all.transpose()[i]

    #T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()

    t = np.linspace(0, 15*(len(GFP.result)-1), len(GFP.result))


    colors = bokeh.palettes.d3["Category10"][3]

    # Populate glyphs
    p.line(
        t/60, GFP.result, line_width=4, color=viridis(51)[color_num],alpha=0.8
    )
    vline1 = Span(location=3, dimension='height', line_color='black', line_width=1,line_dash='dashed')
    p.add_layout(vline1)


color_mapper = LogColorMapper(palette="Viridis256", low=10**condition_low_log, high=10**condition_high_log)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)
p.add_layout(color_bar, 'right')



show(p)

In [26]:
a = -2
10**a

0.01

#experiment 
T7_DNA.ic = 1
T7_DNA.lc = 1
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list

for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#DR = 0.2

#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
GFP.result3 = GFP.result